# <span style="font-size:larger;font-weight:bold">Data Cleaning and grouping New York State Park Trails</span>

In [275]:
import pandas as pd
import geopandas as gpd
import requests
import json
from pyproj import Geod
from shapely.geometry import Polygon, LineString, MultiLineString, Point, GeometryCollection
from shapely import wkt
import matplotlib.pyplot as plt
import shapely.geometry
from shapely.ops import unary_union
import seaborn as sns
import numpy as np
from shapely.wkt import loads
from math import radians, cos, sin, sqrt
import re
import folium
import requests
import urllib.parse
import urllib.request
import urllib3

In [274]:
class GeoDataProcessor:
    """
    A class for processing geospatial data and merging with park information.

    Attributes:
        gdf (GeoDataFrame): a GeoDataFrame containing informations about trails in the New York State Parks.
        df_NYS (DataFrame): a DataFrame containing information about parks in New York State.
    
    Methods:
        read_data: a method to read data from a url in a panda dataframe.
        convert_geomretry: a method that converts the geometry column in a DataFrame from WKT format 
                           to a GeoDataFrame with a proper geometry column. 
        clean_data: a method that fills missing values and replaces incorrect data with correct values in several columns of the
                    GeoDataFrame gdf. It then filters the relevant columns and replaces missing trail names with the value 'Missing Trail Name'.
        group_data(gdf_filtered): a method to group data by the columns 'group_cols' and aggregates the geometry of trail paths using 
                                  the unary_union method and sums the length of the trails. 
        clean_name(name): a method to match the names of parks in the df_NYS DataFrame with processed gdf DataFrame.
        load_NYS_data(filename): a method to load the park information from a csv file, match the park names using clean_name method,
                                 and drop specific rows.
        merge_NYS_data(): a method to merge the processed gdf and df_NYS DataFrames.
    """
    def __init__(self, url, group_cols):
        """
        Initializes the url containg the trail path data.
        Initializes the group of columns that will be selected from the dataset to group the data.
        """
        self.url = url
        self.group_cols = group_cols
        
    def read_data(self):
        self.df = pd.read_csv(self.url)
        
    def convert_geometry(self, df):
        self.df['the_geom'] = self.df['the_geom'].apply(lambda x: loads(x))
        self.gdf = gpd.GeoDataFrame(self.df, geometry='the_geom', crs='EPSG:4326')
        self.gdf.rename(columns={'the_geom': 'geometry'}, inplace=True)
        
    def clean_data(self, gdf):
        self.gdf["Bike"].fillna(method="ffill", inplace=True)
        self.gdf["Snowmb"].fillna(method="ffill", inplace=True)
        self.gdf["Foot"].fillna(method="ffill", inplace=True)
        self.gdf["Horse"].fillna(method="ffill", inplace=True)
        self.gdf["Accessible"].fillna(method="ffill", inplace=True)
        self.gdf["Snowmb"].fillna(method="ffill", inplace=True)
        self.gdf["Facility"].fillna(method="ffill", inplace=True)
        self.gdf['Foot'].replace(['y', 'I', '-99', 'U'], 'Y', inplace=True)
        self.gdf['Bike'].replace(['N`', 'n', 'U', 'I', 'y'], ['N','N','N','Y', 'Y'], inplace=True)
        self.gdf['Horse'].replace(['n', 'U', 'y', 'I'], ['N', 'N', 'Y', 'Y'], inplace=True)
        self.gdf['Snowmb'].replace(['n'], ['N'], inplace=True)
        self.gdf['Accessible'].replace(['y', 'n'], ['Y', 'N'], inplace=True)
        self.gdf['Condition'].replace(['good', 'GOod','fair', 'poor'], ['Good', 'Good', 'Fair', 'Poor'], inplace=True)
        self.gdf.Accessible.unique()
        self.gdf_filtered=self.gdf[['geometry', 'Unit', 'Facility', 'Name', 'Asset', 'Foot', 'Horse', 'Bike', 'Snowmb', 'Accessible', 'Shape_Leng']].copy()
        self.gdf.set_geometry("geometry")
        self.gdf_filtered.loc[self.gdf_filtered['Name'].isna(), 'Name'] = 'Missing Trail Name'
        
    def group_data(self, gdf_filtered):
        self.gdf_grouped = self.gdf_filtered.groupby(self.group_cols, as_index=False).agg({'geometry': unary_union, 'Shape_Leng': 'sum'})
        self.gdf_grouped.set_geometry("geometry", inplace=True)
        self.gdf_grouped.set_crs(epsg=4326, inplace=True)
        return self.gdf_grouped
    
    def clean_name(self, name):
        self.name = name.lower()
        remove_list = ['state historic site', 'campground - thacher state park', 'state park', 'state marine park', 'state historic park', 'state recreation area', 'recreation area']
        for item in remove_list:
            if item in self.name:
                self.name = self.name.split(item)[0].strip()
        self.name = self.name.split('(')[0].strip()
        return self.name.title()
    
    def load_NYS_data(self, filename):
        self.df_NYS = pd.read_csv(filename)
        self.df_NYS['clean_name'] = self.df_NYS['site_name'].apply(self.clean_name)
        self.df_NYS.drop([39,146,147, 159, 182, 211], inplace = True)
        return self.df_NYS
    
    def clean_facility(self, gdf_grouped):
        self.gdf_grouped['clean_facility'] = self.gdf_grouped['Facility'].apply(self.clean_name)
        return self.gdf_grouped
    
    def merge_data(self, gdf_grouped, df_NYS):
        df_merged = pd.merge(gdf_grouped, df_NYS, how='left', left_on='clean_facility', right_on='clean_name')
        return df_merged

In [257]:
#Url for the data
url = "https://data.ny.gov/api/views/7gkb-pzs9/rows.csv?accessType=DOWNLOAD"
#Columns that are used to group the data
group_cols = ['Unit', 'Facility', 'Name', 'Asset', 'Foot', 'Horse', 'Bike', 'Snowmb', 'Accessible'] 
#Create instance for the GeoDataProcessor class
processor = GeoDataProcessor(url, group_cols)
#Calling the method read_data on the processor object to read the data into a dataframe
df = processor.read_data()
#Converting the data into a geopanda dataframe
gdf = processor.convert_geometry(df)
#Cleaning the data
gdf_filtered = processor.clean_data(gdf)
#Grouping the trails by the trail name
gdf_grouped = processor.group_data(gdf_filtered)
#Adding a 'clean_facility' column to match 'Facility' column
gdf_grouped = processor.clean_facility(gdf_grouped)
#Loading webscraped data and adding a 'clean_name' column that will match with the 'clean_facility' column
df_NYS=processor.load_NYS_data("Webscraped_NYS.csv")

In [258]:
df_NYS

,site_name,website,latitude,longitude,address,phone,description,amenities,clean_name
0,Bennington Battlefield State Historic Site,https://parks.ny.gov/historic-sites/12,42.933758,-73.304878,"5157 Route 67Walloomsac, NY 12090",(518) 860-9094,2022 Geocache challengeBennington Battlefield ...,Picnic Area,Bennington Battlefield
1,Clermont State Historic Site,https://parks.ny.gov/historic-sites/16,42.085087,-73.911835,"One Clermont AvenueGermantown, NY 12526",(518) 537-4240,Clermont State Historic Site was the Hudson Ri...,"Equestrian Trails, Gift Shop, Hiking, Picnic A...",Clermont
2,Clinton House State Historic Site,https://parks.ny.gov/historic-sites/1,41.700333,-73.915993,"549 Main StreetPoughkeepsie, NY 12602",(845) 471-1630,The vernacular stone house now known as Clinto...,NaN,Clinton House
3,Crailo State Historic Site,https://parks.ny.gov/historic-sites/30,42.634590,-73.749496,"9 1/2 Riverside AvenueRensselaer, NY 12144",(518) 463-8738,2022 Geocache challengeCrailo is the museum of...,Visitor Center,Crailo
4,Crown Point State Historic Site,https://parks.ny.gov/historic-sites/34,44.024853,-73.424377,"21 Grandview DriveCrown Point, NY 12928",(518) 597-4666,2022 Geocache challengeCrown Point State Histo...,"Hiking, Picnic Area, Snowshoeing/X-Country Ski...",Crown Point
...,...,...,...,...,...,...,...,...,...
226,Whirlpool State Park,https://parks.ny.gov/parks/105,43.119640,-79.062302,"Niagara Scenic ParkwayNiagara Falls, NY 14303",(716) 284-4691,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Hiking, Pavilions and Shelter Rentals...",Whirlpool
227,Wildwood State Park,https://parks.ny.gov/parks/68,40.962742,-72.808426,"790 Hulse Landing RoadWading River, NY 11792",(631) 929-4314,Wildwood State Park comprises 600 acres of und...,"Camper Assistance Program, Campsites, Cabins &...",Wildwood
228,Wilson-Tuscarora State Park,https://parks.ny.gov/parks/69,43.307045,-78.852936,"3371 Lake RoadWilson, NY 14172",(716) 751-6361,Know Before You Go...More InfoPet PolicyA maxi...,"Disc Golf, Fishing, Hiking, Hunting, Marina, P...",Wilson-Tuscarora
229,Wonder Lake State Park,https://parks.ny.gov/parks/190,41.476602,-73.646826,"380 Ludingtonville RoadHolmes, NY 12531",(845) 225-7207,Know Before You Go...More InfoPet PolicyDogs a...,"Hiking, Hunting",Wonder Lake


## Handling Exceptions and Merging Data

In the previous section, I matched the names of the parks in **gdf_grouped** to those in **df_NYS** to enable merging the two dataframes. However, due to minor variations in the names of some parks, I needed to remove certain parts of the name from one dataframe to ensure a successful match. 

This process was followed by a left-join merge of the two dataframes to identify and handle any exceptions and ensure completeness of the merged dataset.

In [259]:
def correct_facility_names(df, name_corrections):
    for old_name, new_name in name_corrections.items():
        if 'clean_facility' in df.columns:
            df.loc[df['clean_facility'] == old_name, 'clean_facility'] = new_name
        if 'clean_name' in df.columns:
            df.loc[df['clean_name'] == old_name, 'clean_name'] = new_name

In [260]:
name_corrections = {'Robert V Riddell': 'Robert V. Riddell',
                    'Robert Treman': 'Robert H. Treman',
                    'Schunnemunk Mountain': 'Schunnemunk',
                    'Max V Shaul': 'Max V. Shaul',
                    "Thompson'S Lake": "Thompson's Lake",
                    'John Boyd Thacher': 'Thacher',
                    'Earl W. Brydges Artpark': 'Artpark'}
correct_facility_names(gdf_grouped, name_corrections)
correct_facility_names(df_NYS, name_corrections)

In [261]:
df_NYS.loc[df_NYS['clean_name'].duplicated(keep=False), 'clean_name']

Series([], Name: clean_name, dtype: object)

In [262]:
df_NYS['clean_name'].unique()

array(['Bennington Battlefield', 'Clermont', 'Clinton House', 'Crailo',
       'Crown Point', 'Darwin Martin House', 'Fort Montgomery',
       'Fort Ontario', 'Ganondagan', 'Grant Cottage', 'Herkimer Home',
       'Hyde Hall', 'Jay Heritage Center', 'John Brown Farm',
       'John Burroughs Memorial', 'John Jay Homestead', 'Johnson Hall',
       "Knox'S Headquarters", 'Lorenzo',
       'National Purple Heart Hall Of Honor', 'New Windsor Cantonment',
       'Olana', 'Old Fort Niagara', 'Oriskany Battlefield',
       'Philipse Manor Hall', 'Planting Fields Arboretum',
       'Sackets Harbor Battlefield', 'Schoharie Crossing',
       'Schuyler Mansion', 'Senate House', 'Sonnenberg Gardens & Mansion',
       'Staatsburgh', 'Steuben Memorial', 'Stonewall Inn',
       'Stony Point Battlefield', 'Walt Whitman Birthplace',
       "Washington'S Headquarters", 'Allan H. Treman', 'Allegany',
       'Anthony Wayne', 'Artpark', 'Barcelona Lighthouse',
       'Battle Island', 'Bayard Cutting Arboret

In [263]:
#Merging both these data frames
df_merged = processor.merge_data(gdf_grouped, df_NYS)
df_merged

,Unit,Facility,Name,Asset,Foot,Horse,Bike,Snowmb,Accessible,geometry,...,clean_facility,site_name,website,latitude,longitude,address,phone,description,amenities,clean_name
0,Allegany,Allegany,ASP 1,Paved Road,Y,Y,N,Y,N,"MULTILINESTRING ((-78.78111 42.04172, -78.7810...",...,Allegany,Allegany State Park - Quaker Area,https://parks.ny.gov/parks/1,42.038990,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",Allegany
1,Allegany,Allegany,ASP1,Paved Road,Y,N,Y,Y,N,"MULTILINESTRING ((-78.82620 42.01107, -78.8261...",...,Allegany,Allegany State Park - Quaker Area,https://parks.ny.gov/parks/1,42.038990,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",Allegany
2,Allegany,Allegany,Bay State Rd,Paved Road,Y,Y,N,N,N,"MULTILINESTRING ((-78.80375 42.08613, -78.8039...",...,Allegany,Allegany State Park - Quaker Area,https://parks.ny.gov/parks/1,42.038990,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",Allegany
3,Allegany,Allegany,Bay State Rd,Paved Road,Y,Y,N,Y,N,"LINESTRING (-78.79718 42.08226, -78.79780 42.0...",...,Allegany,Allegany State Park - Quaker Area,https://parks.ny.gov/parks/1,42.038990,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",Allegany
4,Allegany,Allegany,Bay State Rd,Unpaved Road,Y,Y,N,Y,N,"LINESTRING (-78.80453 42.08277, -78.80502 42.0...",...,Allegany,Allegany State Park - Quaker Area,https://parks.ny.gov/parks/1,42.038990,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",Allegany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,Thousand Islands,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,N,N,"LINESTRING (-75.47458 43.70289, -75.47467 43.7...",...,Whetstone Gulf,Whetstone Gulf State Park,https://parks.ny.gov/parks/92,43.702629,-75.459938,"6065 West Rd.Lowville, NY 13367",(315) 376-6630,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Grills, Hiking, Hunting, Pavilions an...",Whetstone Gulf
1878,Thousand Islands,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,Y,N,"LINESTRING (-75.46535 43.70287, -75.46542 43.7...",...,Whetstone Gulf,Whetstone Gulf State Park,https://parks.ny.gov/parks/92,43.702629,-75.459938,"6065 West Rd.Lowville, NY 13367",(315) 376-6630,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Grills, Hiking, Hunting, Pavilions an...",Whetstone Gulf
1879,Thousand Islands,Whetstone Gulf,Observation Tower Trail,Marked Trail,Y,N,N,N,N,"LINESTRING (-75.47319 43.70077, -75.47316 43.7...",...,Whetstone Gulf,Whetstone Gulf State Park,https://parks.ny.gov/parks/92,43.702629,-75.459938,"6065 West Rd.Lowville, NY 13367",(315) 376-6630,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Grills, Hiking, Hunting, Pavilions an...",Whetstone Gulf
1880,Thousand Islands,Whetstone Gulf,South Rim Gorge Trail,Marked Trail,Y,N,N,N,N,"MULTILINESTRING ((-75.47319 43.70077, -75.4732...",...,Whetstone Gulf,Whetstone Gulf State Park,https://parks.ny.gov/parks/92,43.702629,-75.459938,"6065 West Rd.Lowville, NY 13367",(315) 376-6630,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Grills, Hiking, Hunting, Pavilions an...",Whetstone Gulf


In [267]:
df_merged[df_merged['clean_name'].isnull()]

,Unit,Facility,Name,Asset,Foot,Horse,Bike,Snowmb,Accessible,geometry,...,clean_facility,site_name,website,latitude,longitude,address,phone,description,amenities,clean_name


In [265]:
def set_values(df, start_index, end_index, columns, values):
    df.loc[start_index:end_index, columns] = values
    df.fillna('None', inplace=True)

In [266]:
set_values(df_merged, 1397, 1402, ['site_name', 'website', 'latitude', 'longitude'], 
           ['Mohawk River State Park', 'https://www.niskayuna.org/parks/pages/mohawk-river-state-park', '42.8036', '-73.8515'])
set_values(df_merged, 720, 728, ['site_name', 'website', 'latitude', 'longitude'], 
           ['Hither Woods Preserve','None', '41.0416667', '-71.9902778'])
set_values(df_merged, 729, 748, ['site_name', 'website', 'latitude', 'longitude'], 
           ['Lee Koppelman Park Preserve','None', '41.0297222', '-71.9872222'])
set_values(df_merged, 523, 523, ['site_name', 'website', 'latitude', 'longitude'], 
           ['Amsterdam Beach Preserve','None', ' 42.871993', '-74.442045'])

In [271]:
df_merged[df_merged['site_name']=='Mohawk River State Park']

,Unit,Facility,Name,Asset,Foot,Horse,Bike,Snowmb,Accessible,geometry,...,clean_facility,site_name,website,latitude,longitude,address,phone,description,amenities,clean_name
1397,Saratoga,Mohawk River,Blue Trail,Marked Trail,Y,N,N,N,N,"MULTILINESTRING ((-73.85198 42.80293, -73.8519...",...,Mohawk River,Mohawk River State Park,https://www.niskayuna.org/parks/pages/mohawk-r...,42.8036,-73.8515,None,None,None,None,None
1398,Saratoga,Mohawk River,Green Trail,Marked Trail,Y,N,N,N,N,"MULTILINESTRING ((-73.85664 42.80218, -73.8565...",...,Mohawk River,Mohawk River State Park,https://www.niskayuna.org/parks/pages/mohawk-r...,42.8036,-73.8515,None,None,None,None,None
1399,Saratoga,Mohawk River,Purple Trail,Marked Trail,Y,N,N,N,N,"MULTILINESTRING ((-73.85181 42.80129, -73.8517...",...,Mohawk River,Mohawk River State Park,https://www.niskayuna.org/parks/pages/mohawk-r...,42.8036,-73.8515,None,None,None,None,None
1400,Saratoga,Mohawk River,Red Trail,Marked Trail,Y,N,N,N,N,"MULTILINESTRING ((-73.85315 42.80246, -73.8531...",...,Mohawk River,Mohawk River State Park,https://www.niskayuna.org/parks/pages/mohawk-r...,42.8036,-73.8515,None,None,None,None,None
1401,Saratoga,Mohawk River,White Trail,Marked Trail,Y,N,N,N,N,"MULTILINESTRING ((-73.85107 42.80053, -73.8509...",...,Mohawk River,Mohawk River State Park,https://www.niskayuna.org/parks/pages/mohawk-r...,42.8036,-73.8515,None,None,None,None,None
1402,Saratoga,Mohawk River,Yellow Trail,Marked Trail,Y,N,N,N,N,"MULTILINESTRING ((-73.85481 42.80370, -73.8548...",...,Mohawk River,Mohawk River State Park,https://www.niskayuna.org/parks/pages/mohawk-r...,42.8036,-73.8515,None,None,None,None,None


In [237]:
df_merged[df_merged['clean_name'].isnull()]['clean_facility'].unique()

array([], dtype=object)

In [273]:
df_merged.geometry.dtype

In [272]:
# save dataframe to a CSV file
df_merged.to_csv('Finalized_Trail_paths.csv')

## Querying Elevation Data for the trails
Now that we have grouped and finalized our trail paths, we would need elevation gains of this trails to access the difficulty of this trail paths. We are using USGS **Elevation Point Query Service** API to find the elevations for coordinates of the trail. However, since each trail path can have over 800 coordinates, the computation time can be very large.

In [481]:
class ElevationCalculator:
    '''
    A class for calculating elevation gain and angle of ascent/descent for geospatial data.
    
    Method:
    make_remote_request: a helper function for making a remote HTTP GET request and handling errors that may occur.
    get_elevation : a helper function for retrieving elevation data from the USGS Elevation Point Query Service.
    calculate_elevation : Main function for calculating elevation data by retrieving elevation data for each point in the input DataFrame 
                          and storing it in the output DataFrame.
    calculate_elevation_gain: function for calculating elevation gain for each trails.
    calculate_angle_of_descent : function for calculating angle of ascent/descent for each trail
    difficulty : function for calculating Yoshemite Decimal System Rating (YDR) for each trail
    '''
    def __init__(self, data):
        '''
        Constructor for the ElevationCalculator class. Initializes instance variables and performs necessary preprocessing.
        
        Parameters:
        - data: Pandas DataFrame containing geospatial data to be processed
        
        '''
        self.data = data
        self.df_exploded = self.data.explode(index_parts=False)
        self.df_exploded.index
        self.df_exploded['simplified_geometry'] = self.df_exploded['geometry'].apply(lambda x: x.simplify(0.001, preserve_topology=True))
        self.simplified = self.df_exploded.groupby(['Unit', 'site_name', 'Facility', 'Name', 'Asset', 'Foot', 'Horse', 'Bike', 'Snowmb', 'Accessible','Shape_Leng'])['simplified_geometry'].apply(unary_union).reset_index(name='new_geometry')
        self.simplified['Coords'] = None
        self.simplified['Height'] = None

    def make_remote_request(self, url, params):
        count = 1
        while True:
            try:
                response = requests.get((url + urllib.parse.urlencode(params)))
            except (OSError, urllib3.exceptions.ProtocolError) as error:
                print('\n')
                print('*' * 20, 'Error Occured', '*' * 20)
                print(f'Number of tries: {count}')
                print(f'URL: {url}')
                print(error)
                print('\n')
                count += 1
                continue
            break

        return response

    def get_elevation(self, lat_lon_pairs):
        url = 'https://epqs.nationalmap.gov/v1/json?'
        elevations = []    
        for lat, lon in lat_lon_pairs:
            params = {'x': lon,
                      'y': lat,
                      'wkid': '4326',
                      'units': 'Meters',
                      'includeDate': 'false'}
            response = self.make_remote_request(url, params)
            elevation = response.json()['value']
            #print(elevation)
            elevations.append(elevation)
        return elevations

    def calculate_elevation(self):

        o=self.simplified.to_json()
        self.geo_j_object = json.loads(o)
        for feature in self.geo_j_object["features"]:
            empty_list=[]
            if feature['geometry']['type'] == 'MultiLineString':
                    for line in feature['geometry']['coordinates']:
                        empty_list.extend(line)
                    self.simplified.iloc[int(feature['id']), self.simplified.columns.get_loc('Coords')] = [empty_list]
            elif feature['geometry']['type'] == 'LineString': 
                    empty_list.extend(feature['geometry']['coordinates'])
                    self.simplified.iloc[int(feature['id']), self.simplified.columns.get_loc('Coords')] = [empty_list]
        self.simplified['Coords'] = self.simplified['Coords'].apply(lambda x: [c for coord in x for c in coord])

        for i, row in self.simplified.iterrows():
            coordinates = row['Coords']
            lat_lon_pairs = [(lat, lon) for lon, lat in coordinates]
            elevations = self.get_elevation(lat_lon_pairs)
            #print(elevations, i)
            self.simplified.at[i, 'Height'] = np.array(elevations)
            print(i+1)
        return self.simplified
    
    def calculate_elevation_gain(self, row):
        heights = row['Height']
        heights = heights.astype(float)
        elevation_gain = max(heights) - min(heights)
        return elevation_gain
    
    def calculate_angle_of_descent(self, row):
        elevation_gain = row['Elevation_Gain']
        distance = row['Shape_Leng']
        angle_of_descent = np.arctan(elevation_gain / distance) * 180 / np.pi
        return angle_of_descent
    def difficulty(self, row):
        elevation_gain = row['Elevation_Gain']
        distance = row['Shape_Leng']
        type_fac = row['type_factor']
        if elevation_gain == 0: 
            YDR = (0.6 * np.log10(distance*0.000621371)) + type_fac
        else:
            YDR = (0.6 * np.log10(distance*0.000621371) + 0.4 * np.log10(elevation_gain)) + type_fac
        return YDR

In [473]:
# Create an instance of the ElevationCalculator class with the merged dataframe as input
Calculator = ElevationCalculator(df_merged)

In [336]:
# Calculate the elevation data for the simplified geometry in the dataframe
#Each trail path is counted to see the progress
df_elevation = Calculator.calculate_elevation()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187


******************** Error Occured ********************
Number of tries: 1
URL: https://epqs.nationalmap.gov/v1/json?
('Connection aborted.', OSError(22, 'Invalid argument'))




******************** Error Occured ********************
Number of tries: 1
URL: https://epqs.nationalmap.gov/v1/json?
('Connection aborted.', OSError(22, 'Invalid argument'))






In [337]:
df_elevation

,Unit,site_name,Facility,Name,Asset,Foot,Horse,Bike,Snowmb,Accessible,Shape_Leng,new_geometry,Coords,Height
0,Allegany,Allegany State Park - Quaker Area,Allegany,ASP 1,Paved Road,Y,Y,N,Y,N,2762.553239,"MULTILINESTRING ((-78.78111 42.04172, -78.7804...","[[-78.78111252995613, 42.04171639655458], [-78...","[662.329956055, 679.899963379, 682.999938965, ..."
1,Allegany,Allegany State Park - Quaker Area,Allegany,ASP1,Paved Road,Y,N,Y,Y,N,11247.667588,"MULTILINESTRING ((-78.82620 42.01107, -78.8219...","[[-78.82619908330466, 42.011072025742386], [-7...","[452.929992676, 488.270050049, 498.879974365, ..."
2,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,N,N,1599.398857,"MULTILINESTRING ((-78.80375 42.08613, -78.8033...","[[-78.80375131191295, 42.08612775274329], [-78...","[426.910034180, 420.330017090, 415.729980469, ..."
3,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,Y,N,722.157056,"LINESTRING (-78.79718 42.08226, -78.80375 42.0...","[[-78.7971839554472, 42.082256952313976], [-78...","[447.870086670, 426.910034180]"
4,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Unpaved Road,Y,Y,N,Y,N,5830.772003,"LINESTRING (-78.80453 42.08277, -78.80787 42.0...","[[-78.80453235289862, 42.082769915085855], [-7...","[434.500030518, 444.890014648, 473.770019531, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,N,N,3378.218496,"LINESTRING (-75.47458 43.70289, -75.48552 43.6...","[[-75.47457947947044, 43.70288641304755], [-75...","[502.876861572, 547.890380859, 558.764099121, ..."
1878,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,Y,N,1032.669680,"LINESTRING (-75.46535 43.70287, -75.46734 43.7...","[[-75.46535042618731, 43.702873326877224], [-7...","[407.938537598, 428.315673828, 446.547027588, ..."
1879,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,Observation Tower Trail,Marked Trail,Y,N,N,N,N,273.318398,"LINESTRING (-75.47319 43.70077, -75.47111 43.7...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 458.813781738]"
1880,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,South Rim Gorge Trail,Marked Trail,Y,N,N,N,N,4268.759962,"MULTILINESTRING ((-75.47319 43.70077, -75.4725...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 520.354003906, 509.490600586, ..."


In [396]:
# Calculate the elevation gain for each row in the dataframe using the 'calculate_elevation_gain' method
# and add the results as a new column 'Elevation_Gain'
df_elevation['Elevation_Gain'] = df_elevation.apply(Calculator.calculate_elevation_gain, axis=1)

In [398]:
df_elevation

,Unit,site_name,Facility,Name,Asset,Foot,Horse,Bike,Snowmb,Accessible,Shape_Leng,new_geometry,Coords,Height,Elevation_Gain,Angle_of_Descent
0,Allegany,Allegany State Park - Quaker Area,Allegany,ASP 1,Paved Road,Y,Y,N,Y,N,2762.553239,"MULTILINESTRING ((-78.78111 42.04172, -78.7804...","[[-78.78111252995613, 42.04171639655458], [-78...","[662.329956055, 679.899963379, 682.999938965, ...",38.980042,0.808398
1,Allegany,Allegany State Park - Quaker Area,Allegany,ASP1,Paved Road,Y,N,Y,Y,N,11247.667588,"MULTILINESTRING ((-78.82620 42.01107, -78.8219...","[[-78.82619908330466, 42.011072025742386], [-7...","[452.929992676, 488.270050049, 498.879974365, ...",230.279968,1.172886
2,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,N,N,1599.398857,"MULTILINESTRING ((-78.80375 42.08613, -78.8033...","[[-78.80375131191295, 42.08612775274329], [-78...","[426.910034180, 420.330017090, 415.729980469, ...",18.770050,0.672375
3,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,Y,N,722.157056,"LINESTRING (-78.79718 42.08226, -78.80375 42.0...","[[-78.7971839554472, 42.082256952313976], [-78...","[447.870086670, 426.910034180]",20.960052,1.662499
4,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Unpaved Road,Y,Y,N,Y,N,5830.772003,"LINESTRING (-78.80453 42.08277, -78.80787 42.0...","[[-78.80453235289862, 42.082769915085855], [-7...","[434.500030518, 444.890014648, 473.770019531, ...",253.869904,2.493065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,N,N,3378.218496,"LINESTRING (-75.47458 43.70289, -75.48552 43.6...","[[-75.47457947947044, 43.70288641304755], [-75...","[502.876861572, 547.890380859, 558.764099121, ...",65.474701,1.110335
1878,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,Y,N,1032.669680,"LINESTRING (-75.46535 43.70287, -75.46734 43.7...","[[-75.46535042618731, 43.702873326877224], [-7...","[407.938537598, 428.315673828, 446.547027588, ...",94.932037,5.252367
1879,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,Observation Tower Trail,Marked Trail,Y,N,N,N,N,273.318398,"LINESTRING (-75.47319 43.70077, -75.47111 43.7...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 458.813781738]",50.888458,10.546989
1880,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,South Rim Gorge Trail,Marked Trail,Y,N,N,N,N,4268.759962,"MULTILINESTRING ((-75.47319 43.70077, -75.4725...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 520.354003906, 509.490600586, ...",161.962860,2.172842


In [380]:
df_elevation.Elevation_Gain.dtype

dtype('float64')

In [484]:
# Calculate the angle of descent for each row in the dataframe using the 'calculate_angle_of_descent' method
# and add the results as a new column 'Angle_of_Descent'
df_elevation['Angle_of_Descent'] = df_elevation.apply(Calculator.calculate_angle_of_descent, axis=1)

In [485]:
#df_elevation.drop('angle_of_descent', axis = 1, inplace=True)
df_elevation

,Unit,site_name,Facility,Name,Asset,Foot,Horse,Bike,Snowmb,Accessible,Shape_Leng,new_geometry,Coords,Height,Elevation_Gain,Angle_of_Descent,type_factor,YDR
0,Allegany,Allegany State Park - Quaker Area,Allegany,ASP 1,Paved Road,Y,Y,N,Y,N,2762.553239,"MULTILINESTRING ((-78.78111 42.04172, -78.7804...","[[-78.78111252995613, 42.04171639655458], [-78...","[662.329956055, 679.899963379, 682.999938965, ...",38.980042,0.808398,0.0,0.777134
1,Allegany,Allegany State Park - Quaker Area,Allegany,ASP1,Paved Road,Y,N,Y,Y,N,11247.667588,"MULTILINESTRING ((-78.82620 42.01107, -78.8219...","[[-78.82619908330466, 42.011072025742386], [-7...","[452.929992676, 488.270050049, 498.879974365, ...",230.279968,1.172886,0.0,1.451551
2,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,N,N,1599.398857,"MULTILINESTRING ((-78.80375 42.08613, -78.8033...","[[-78.80375131191295, 42.08612775274329], [-78...","[426.910034180, 420.330017090, 415.729980469, ...",18.770050,0.672375,0.0,0.507771
3,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,Y,N,722.157056,"LINESTRING (-78.79718 42.08226, -78.80375 42.0...","[[-78.7971839554472, 42.082256952313976], [-78...","[447.870086670, 426.910034180]",20.960052,1.662499,0.0,0.319747
4,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Unpaved Road,Y,Y,N,Y,N,5830.772003,"LINESTRING (-78.80453 42.08277, -78.80787 42.0...","[[-78.80453235289862, 42.082769915085855], [-7...","[434.500030518, 444.890014648, 473.770019531, ...",253.869904,2.493065,0.0,1.397291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,N,N,3378.218496,"LINESTRING (-75.47458 43.70289, -75.48552 43.6...","[[-75.47457947947044, 43.70288641304755], [-75...","[502.876861572, 547.890380859, 558.764099121, ...",65.474701,1.110335,0.1,1.019653
1878,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,Y,N,1032.669680,"LINESTRING (-75.46535 43.70287, -75.46734 43.7...","[[-75.46535042618731, 43.702873326877224], [-7...","[407.938537598, 428.315673828, 446.547027588, ...",94.932037,5.252367,0.1,0.775353
1879,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,Observation Tower Trail,Marked Trail,Y,N,N,N,N,273.318398,"LINESTRING (-75.47319 43.70077, -75.47111 43.7...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 458.813781738]",50.888458,10.546989,0.1,0.320660
1880,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,South Rim Gorge Trail,Marked Trail,Y,N,N,N,N,4268.759962,"MULTILINESTRING ((-75.47319 43.70077, -75.4725...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 520.354003906, 509.490600586, ...",161.962860,2.172842,0.1,1.237958


In [482]:
df_elevation['type_factor'] = df_elevation['Asset'].map({'Marked Trail': 0.1, 'Unmarked Trail': 0.3, 'Sidewalk': 0, 'Paved Road': 0, 
                                 'Unpaved Road': 0.0, 'Pathway': 0, 'Carriage Road': 0, 'new': 0, 'Digitized Trail':0.1,
                                 'Plowed Road': 0.0, 'Detour': 0, 'Historic Path': 0.0, 'Bike Path': 0.0, 
                                 'Pedestrian Bridge': 0, 'Historic Walkway': 0, 'Bridge Walkway': 0})

In [483]:
df_elevation

,Unit,site_name,Facility,Name,Asset,Foot,Horse,Bike,Snowmb,Accessible,Shape_Leng,new_geometry,Coords,Height,Elevation_Gain,Angle_of_Descent,type_factor,YDR
0,Allegany,Allegany State Park - Quaker Area,Allegany,ASP 1,Paved Road,Y,Y,N,Y,N,2762.553239,"MULTILINESTRING ((-78.78111 42.04172, -78.7804...","[[-78.78111252995613, 42.04171639655458], [-78...","[662.329956055, 679.899963379, 682.999938965, ...",38.980042,0.808398,0.0,0.777134
1,Allegany,Allegany State Park - Quaker Area,Allegany,ASP1,Paved Road,Y,N,Y,Y,N,11247.667588,"MULTILINESTRING ((-78.82620 42.01107, -78.8219...","[[-78.82619908330466, 42.011072025742386], [-7...","[452.929992676, 488.270050049, 498.879974365, ...",230.279968,1.172886,0.0,1.451551
2,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,N,N,1599.398857,"MULTILINESTRING ((-78.80375 42.08613, -78.8033...","[[-78.80375131191295, 42.08612775274329], [-78...","[426.910034180, 420.330017090, 415.729980469, ...",18.770050,0.672375,0.0,0.507771
3,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,Y,N,722.157056,"LINESTRING (-78.79718 42.08226, -78.80375 42.0...","[[-78.7971839554472, 42.082256952313976], [-78...","[447.870086670, 426.910034180]",20.960052,1.662499,0.0,0.319747
4,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Unpaved Road,Y,Y,N,Y,N,5830.772003,"LINESTRING (-78.80453 42.08277, -78.80787 42.0...","[[-78.80453235289862, 42.082769915085855], [-7...","[434.500030518, 444.890014648, 473.770019531, ...",253.869904,2.493065,0.0,1.397291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,N,N,3378.218496,"LINESTRING (-75.47458 43.70289, -75.48552 43.6...","[[-75.47457947947044, 43.70288641304755], [-75...","[502.876861572, 547.890380859, 558.764099121, ...",65.474701,1.110335,0.1,1.019653
1878,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,Y,N,1032.669680,"LINESTRING (-75.46535 43.70287, -75.46734 43.7...","[[-75.46535042618731, 43.702873326877224], [-7...","[407.938537598, 428.315673828, 446.547027588, ...",94.932037,5.252367,0.1,0.775353
1879,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,Observation Tower Trail,Marked Trail,Y,N,N,N,N,273.318398,"LINESTRING (-75.47319 43.70077, -75.47111 43.7...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 458.813781738]",50.888458,10.546989,0.1,0.320660
1880,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,South Rim Gorge Trail,Marked Trail,Y,N,N,N,N,4268.759962,"MULTILINESTRING ((-75.47319 43.70077, -75.4725...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 520.354003906, 509.490600586, ...",161.962860,2.172842,0.1,1.237958


In [486]:
df_elevation['DR'] = df_elevation.apply(Calculator.difficulty, axis=1)

In [495]:
#df_elevation.drop('YDR', axis =1, inplace = True)
df_elevation

,Unit,site_name,Facility,Name,Asset,Foot,Horse,Bike,Snowmb,Accessible,Shape_Leng,new_geometry,Coords,Height,Elevation_Gain,Angle_of_Descent,type_factor,DR
0,Allegany,Allegany State Park - Quaker Area,Allegany,ASP 1,Paved Road,Y,Y,N,Y,N,2762.553239,"MULTILINESTRING ((-78.78111 42.04172, -78.7804...","[[-78.78111252995613, 42.04171639655458], [-78...","[662.329956055, 679.899963379, 682.999938965, ...",38.980042,0.808398,0.0,0.777134
1,Allegany,Allegany State Park - Quaker Area,Allegany,ASP1,Paved Road,Y,N,Y,Y,N,11247.667588,"MULTILINESTRING ((-78.82620 42.01107, -78.8219...","[[-78.82619908330466, 42.011072025742386], [-7...","[452.929992676, 488.270050049, 498.879974365, ...",230.279968,1.172886,0.0,1.451551
2,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,N,N,1599.398857,"MULTILINESTRING ((-78.80375 42.08613, -78.8033...","[[-78.80375131191295, 42.08612775274329], [-78...","[426.910034180, 420.330017090, 415.729980469, ...",18.770050,0.672375,0.0,0.507771
3,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Paved Road,Y,Y,N,Y,N,722.157056,"LINESTRING (-78.79718 42.08226, -78.80375 42.0...","[[-78.7971839554472, 42.082256952313976], [-78...","[447.870086670, 426.910034180]",20.960052,1.662499,0.0,0.319747
4,Allegany,Allegany State Park - Quaker Area,Allegany,Bay State Rd,Unpaved Road,Y,Y,N,Y,N,5830.772003,"LINESTRING (-78.80453 42.08277, -78.80787 42.0...","[[-78.80453235289862, 42.082769915085855], [-7...","[434.500030518, 444.890014648, 473.770019531, ...",253.869904,2.493065,0.0,1.297291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,N,N,3378.218496,"LINESTRING (-75.47458 43.70289, -75.48552 43.6...","[[-75.47457947947044, 43.70288641304755], [-75...","[502.876861572, 547.890380859, 558.764099121, ...",65.474701,1.110335,0.1,1.019653
1878,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,North Rim Gorge Tail,Marked Trail,Y,N,N,Y,N,1032.669680,"LINESTRING (-75.46535 43.70287, -75.46734 43.7...","[[-75.46535042618731, 43.702873326877224], [-7...","[407.938537598, 428.315673828, 446.547027588, ...",94.932037,5.252367,0.1,0.775353
1879,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,Observation Tower Trail,Marked Trail,Y,N,N,N,N,273.318398,"LINESTRING (-75.47319 43.70077, -75.47111 43.7...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 458.813781738]",50.888458,10.546989,0.1,0.320660
1880,Thousand Islands,Whetstone Gulf State Park,Whetstone Gulf,South Rim Gorge Trail,Marked Trail,Y,N,N,N,N,4268.759962,"MULTILINESTRING ((-75.47319 43.70077, -75.4725...","[[-75.47318918623482, 43.70077126913577], [-75...","[509.702239990, 520.354003906, 509.490600586, ...",161.962860,2.172842,0.1,1.237958


In [496]:
#Merging the park and trail elevation data for the finalized dataset
group_col=['Unit', 'site_name', 'Facility', 'Name', 'Asset', 'Foot', 'Horse', 'Bike', 'Snowmb', 'Accessible','Shape_Leng']
gdf_trails_NYS = pd.merge(df_merged, df_elevation, how='inner', left_on=group_col, right_on=group_col)

In [497]:
#Dropping irrelavant and duplicate columns
gdf_trails_NYS.drop(['Facility', 'clean_facility', 'new_geometry', 'Height', 'Coords', 'clean_name'], axis = 1, inplace = True)

In [498]:
#Creating order of the data columns before saving the data as a csv file
col_order = ['Unit', 'site_name', 'latitude', 'longitude', 'address', 'phone', 'website', 'description', 
             'amenities','Name', 'geometry', 'Asset', 'type_factor', 'Foot', 'Horse', 'Bike', 'Snowmb', 'Accessible',
             'Shape_Leng', 'Elevation_Gain', 'Angle_of_Descent', 'DR' ]
gdf_trails_NYS = gdf_trails_NYS.reindex(columns=col_order)
gdf_trails_NYS.to_csv('Finalized_Trail_paths.csv', index=False)

In [499]:
gdf_trails_NYS

,Unit,site_name,latitude,longitude,address,phone,website,description,amenities,Name,...,type_factor,Foot,Horse,Bike,Snowmb,Accessible,Shape_Leng,Elevation_Gain,Angle_of_Descent,DR
0,Allegany,Allegany State Park - Quaker Area,42.03899,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,https://parks.ny.gov/parks/1,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",ASP 1,...,0.0,Y,Y,N,Y,N,2762.553239,38.980042,0.808398,0.777134
1,Allegany,Allegany State Park - Quaker Area,42.03899,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,https://parks.ny.gov/parks/1,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",ASP1,...,0.0,Y,N,Y,Y,N,11247.667588,230.279968,1.172886,1.451551
2,Allegany,Allegany State Park - Quaker Area,42.03899,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,https://parks.ny.gov/parks/1,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",Bay State Rd,...,0.0,Y,Y,N,N,N,1599.398857,18.770050,0.672375,0.507771
3,Allegany,Allegany State Park - Quaker Area,42.03899,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,https://parks.ny.gov/parks/1,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",Bay State Rd,...,0.0,Y,Y,N,Y,N,722.157056,20.960052,1.662499,0.319747
4,Allegany,Allegany State Park - Quaker Area,42.03899,-78.843833,"2373 ASP, Rte 1Salamanca, NY 14779",(716) 354-2182,https://parks.ny.gov/parks/1,The Quaker area is known for its two lakes and...,"Boat Launches, Camper Assistance Program, Camp...",Bay State Rd,...,0.0,Y,Y,N,Y,N,5830.772003,253.869904,2.493065,1.297291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,Thousand Islands,Whetstone Gulf State Park,43.702629,-75.459938,"6065 West Rd.Lowville, NY 13367",(315) 376-6630,https://parks.ny.gov/parks/92,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Grills, Hiking, Hunting, Pavilions an...",North Rim Gorge Tail,...,0.1,Y,N,N,N,N,3378.218496,65.474701,1.110335,1.019653
1878,Thousand Islands,Whetstone Gulf State Park,43.702629,-75.459938,"6065 West Rd.Lowville, NY 13367",(315) 376-6630,https://parks.ny.gov/parks/92,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Grills, Hiking, Hunting, Pavilions an...",North Rim Gorge Tail,...,0.1,Y,N,N,Y,N,1032.669680,94.932037,5.252367,0.775353
1879,Thousand Islands,Whetstone Gulf State Park,43.702629,-75.459938,"6065 West Rd.Lowville, NY 13367",(315) 376-6630,https://parks.ny.gov/parks/92,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Grills, Hiking, Hunting, Pavilions an...",Observation Tower Trail,...,0.1,Y,N,N,N,N,273.318398,50.888458,10.546989,0.320660
1880,Thousand Islands,Whetstone Gulf State Park,43.702629,-75.459938,"6065 West Rd.Lowville, NY 13367",(315) 376-6630,https://parks.ny.gov/parks/92,Know Before You Go...More InfoPet PolicyA maxi...,"Fishing, Grills, Hiking, Hunting, Pavilions an...",South Rim Gorge Trail,...,0.1,Y,N,N,N,N,4268.759962,161.962860,2.172842,1.237958
